In [1]:
import torch
import torch.nn as nn
from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer, LlamaForCausalLM

import sys, os, json, math
from tqdm import tqdm
import numpy as np

/home/jgryu/miniconda3/envs/Wcomp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### AWQ

In [16]:
def latest_version_path(cache_dir, model_name, branch = 'main'):
    model_name_dir =  "models--" + model_name.replace('/', '--')
    path = os.path.join(cache_dir, model_name_dir)
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

cache_directory = "../Wparam_dataset_v0/model_zoo/huggingface" 
ckpt_path = latest_version_path(cache_directory, 'meta-llama/Meta-Llama-3-8B')
net = LlamaForCausalLM.from_pretrained(ckpt_path, local_files_only=True)

# ckpt_path = '/home/jgryu/Weight_compression/model_cache/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920'
# net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
# state_dict = net.state_dict()

std = 0.012528747320175171

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.57s/it]


In [17]:
bpps = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12]
mses = []
for bpp in bpps:
    ckpt_path = f'../model_cache_reconstructed/awq/llama3-8b-my-w{bpp}-g128-fake-quantized'
    recon_net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
    recon_state_dict = recon_net.state_dict()

    n = 0
    mse = 0
    for k, v in state_dict.items():
        if 'mlp' not in k and 'attn' not in k: continue
        
        mse += ((recon_state_dict[k] - v)**2).sum()
        n += v.numel()
        
    mse = mse / n / std **2 
    mses.append(mse.item())
    print(mse)

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.96s/it]


tensor(23.0156)


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.15s/it]


tensor(21.9569)


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.10s/it]


...

tensor(12.1139)


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.01s/it]


tensor(13.7742)


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.03s/it]


tensor(57.4543)


### AWQ
bpp:  [2, 3, 4, 5, 6, 7, 8, 9, 10, 12]

mse:  [23.015626907348633, 21.956884384155273, 18.508888244628906, 189.52224731445312, 21.462759017944336, 18.203344345092773, 23.026723861694336, 12.113859176635742, 13.774173736572266, 57.454349517822266]

In [8]:
print('bpp: ', bpps)
print('mse: ', mses)

bpp:  [2, 3, 4, 5, 6, 7, 8, 9, 10, 12]
mse:  [23.015626907348633, 21.956884384155273, 18.508888244628906, 189.52224731445312, 21.462759017944336, 18.203344345092773, 23.026723861694336, 12.113859176635742, 13.774173736572266, 57.454349517822266]


### VQ

In [9]:
def latest_version_path(cache_dir, model_name, branch = 'main'):
    model_name_dir =  "models--" + model_name.replace('/', '--')
    path = os.path.join(cache_dir, model_name_dir)
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

cache_directory = "../Wparam_dataset_v0/model_zoo/huggingface" 
ckpt_path = latest_version_path(cache_directory, 'meta-llama/Meta-Llama-3-8B')
net = LlamaForCausalLM.from_pretrained(ckpt_path, local_files_only=True)

# ckpt_path = '/home/jgryu/Weight_compression/model_cache/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920'
# net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
state_dict = net.state_dict()

std = 0.012528747320175171

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.86s/it]


In [14]:
ckpt_paths = [
    # '../model_cache_reconstructed/vq_seedlm_/mlp_16_row_dataset.pt_v101/size16_ne256_de256_P4_batch_size512_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_0.09639_total_iter_2000000.pth.tar',
    # '../model_cache_reconstructed/vq_seedlm_/mlp_16_row_dataset.pt_v101/size16_ne512_de256_P4_batch_size512_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_0.0864_total_iter_2000000.pth.tar',
    # '../model_cache_reconstructed/vq_seedlm_/mlp_16_row_dataset.pt_v101/size16_ne512_de256_P8_batch_size512_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_0.00338_total_iter_1950000.pth.tar',
    # '../model_cache_reconstructed/vq_seedlm_/mlp_16_row_dataset.pt_v101/size16_ne512_de256_P16_batch_size512_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_2e-05_total_iter_1850000.pth.tar'
    '../model_cache_reconstructed/vq_seedlm_/mlp_attn_16_row_dataset.pt/size16_ne256_denc512_P8_KNone_de16_batch_size1024_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_0.01045_total_iter_1850000.pth.tar',
    '../model_cache_reconstructed/vq_seedlm_/mlp_attn_16_row_dataset.pt/size16_ne256_denc512_P16_KNone_de16_batch_size1024_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_0.00029_total_iter_1850000.pth.tar'
]

mses = []
for ckpt_path in ckpt_paths:
    recon_net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
    recon_state_dict = recon_net.state_dict()

    n = 0
    mse = 0
    for k, v in state_dict.items():
        if 'mlp' not in k and 'attn' not in k: continue
        # if 'mlp' not in k: continue
        
        mse += ((recon_state_dict[k] - v)**2).sum()
        n += v.numel()
        
    mse = mse / n / std **2 
    mses.append(mse.item())
    print(mse)

Loading checkpoint shards: 100%|██████████| 7/7 [00:11<00:00,  1.62s/it]


tensor(0.0116)


Loading checkpoint shards: 100%|██████████| 7/7 [00:11<00:00,  1.67s/it]


tensor(0.0004)


In [15]:
# print('bpp: ', bpps)
print('mse: ', mses[-4:])

mse:  [0.011624915525317192, 0.00042339859646745026]


### VQseedlm v101 mlp
bpp:  [6, 6.25, 12.25, 24.25]

<!-- mse:  [0.08586853742599487, 0.07806961983442307, 0.003792528761550784, 3.880444273818284e-05] -->

### VQseedlm v101 mlp attn
bpp:  [12, 24]

mse:  [0.011624915525317192, 0.00042339859646745026]